In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# FastLanguageModel.from_pretrained?

In [3]:
from unsloth import FastLanguageModel
import torch

# fourbit_models = [
#     "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
#     "unsloth/Qwen3-4B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-8B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-14B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-32B-unsloth-bnb-4bit",

#     # 4bit dynamic quants for superior accuracy and low memory use
#     "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     "unsloth/Phi-4",
#     "unsloth/Llama-3.1-8B",
#     "unsloth/Llama-3.2-3B",
#     "unsloth/Qwen3-4B",
#     "unsloth/Qwen3-8B"
#     "unsloth/Qwen3-14B"
#     "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 1024,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = True,    # A bit more accurate, uses 2x memory
    full_finetuning = True, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hoyeol/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-08 17:46:26.025505: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-08 17:46:26.039444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749404786.055654   60624 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749404786.060483   60624 cuda_blas.cc:1407] Unable

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: You selected full finetuning support, but 4bit / 8bit is enabled - disabling LoRA / QLoRA.
==((====))==  Unsloth 2025.6.1: Fast Qwen3 patching. Transformers: 4.52.3.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [4]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,  # Best to choose alpha = rank or rank*2
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,   # We support rank stabilized LoRA
#     loftq_config = None,  # And LoftQ
# )

In [5]:
import pandas as pd
train_caption = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_captions_train.csv", index_col=0)
valid_caption = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_captions_test.csv", index_col=0)
train_path = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_train_filtered.csv", index_col=0)
valid_path = pd.read_csv("/home/hoyeol/projects/GCT634_final/data/DX7_YAMAHA_test_filtered.csv", index_col=0)
train_df = pd.merge(train_path, train_caption[['id', 'caption']], on='id', how='left')
valid_df = pd.merge(valid_path, valid_caption[['id', 'caption']], on='id', how='left')

In [6]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_df[['id', 'caption', 'patch_data']])
valid_dataset = Dataset.from_pandas(valid_df[['id', 'caption', 'patch_data']])

In [7]:
from prompt import zeroshot_prompt
import ast
from utils import serialize_specs
def proprocess(examples, prompt = zeroshot_prompt):
    conversations = []
    for caption, patch_data in zip(examples["caption"], examples["patch_data"]):
        question = prompt.format(prompt=caption)
        specs = ast.literal_eval(patch_data)
        specs.pop('name', None)
        specs.pop('has_fixed_freqs', None)
        solution = '```python\n' + 'specs = ' +serialize_specs(specs) + '```\n'
        conversations.append([
            {"role" : "user",      "content" : question},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [8]:
train_dataset = tokenizer.apply_chat_template(
    train_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
valid_dataset = tokenizer.apply_chat_template(
    valid_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
train_dataset = pd.Series(train_dataset)
valid_dataset = pd.Series(valid_dataset)
train_dataset.name = "text"
valid_dataset.name = "text"
train_dataset = Dataset.from_pandas(pd.DataFrame(train_dataset)).shuffle()
valid_dataset = Dataset.from_pandas(pd.DataFrame(valid_dataset))

Map: 100%|██████████| 63/63 [00:00<00:00, 4175.35 examples/s]


In [9]:
lengths = [len(tokenizer(train_dataset[i]['text'])['input_ids']) for i in range(len(train_dataset))]
print(max(lengths))

599


In [10]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset, # Can set up evaluation!
    args = SFTConfig(
        eval_strategy="steps",
        eval_steps = 10,
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        # warmup_steps = 10,
        warmup_ratio = 0.1,
        num_train_epochs = 3, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        # bf16 = True,
    ),
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Unsloth: Tokenizing ["text"] (num_proc=64): 100%|██████████| 600/600 [00:06<00:00, 99.93 examples/s] 
num_proc must be <= 63. Reducing num_proc to 63 for dataset of size 63.
Map (num_proc=64): 100%|██████████| 600/600 [00:00<00:00, 1111.09 examples/s]
num_proc must be <= 63. Reducing num_proc to 63 for dataset of size 63.
Map (num_proc=63): 100%|██████████| 63/63 [00:00<00:00, 110.95 examples/s]


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 3 | Total steps = 225
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,022,468,096/4,022,468,096 (100.00% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
10,0.381800,0.407652
20,0.326500,0.379948
30,0.444800,0.418058
40,0.481800,0.457081
50,0.374100,0.340942
60,0.359700,0.336258
70,0.324200,0.327588
80,0.295800,0.322566
90,0.298800,0.311529
100,0.256800,0.308080


Unsloth: Will smartly offload gradients to save VRAM!


In [12]:
#0.180400	0.266946

In [16]:
caption = valid_df.iloc[23]['caption']
messages = [
    {"role" : "user", "content" : zeroshot_prompt.format(prompt=caption)}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)
print(caption)

from transformers import TextStreamer
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # temperature = 1.0, #, top_p = 0.8, top_k = 20, # For non thinking
    # do_sample = False,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Trumpet, which has a bright and distinct timbre with a relatively quick attack and moderate sustain.
```python
specs = {
    'modmatrix': [
		[0, 1, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 1, 0, 0],
		[0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 1],
		[0, 0, 0, 0, 0, 1]
    ],
    'outmatrix': [1, 0, 1, 0, 1, 0],
    'feedback': 7,
    'coarse': [1, 1, 2, 1, 1, 1],
    'fine': [0, 0, 0, 0, 0, 0],
    'detune': [0, 0, 0, 0, 0, 0],
    'transpose': 0,
    'ol': [78, 99, 89, 99, 81, 99],
    'eg_rate': [
		[99, 99, 99, 99, 99, 99],
		[99, 99, 99, 99, 99, 99],
		[99, 99, 99, 99, 99, 99],
		[99, 99, 99, 99, 99, 99]
    ],
    'eg_level': [
		[99, 99, 99, 99, 99, 99],
		[99, 99, 99, 99, 99, 99],
		[99, 99, 99, 99, 99, 99],
		[0, 0, 0, 0, 0, 0]
    ],
    'sensitivity': [0, 0, 0, 0, 0, 0],
}```
<|im_end|>


In [17]:
from utils import parse_last_specs, render_from_specs, validate_specs
import IPython.display as ipd
specs = parse_last_specs(tokenizer.decode(output[0], skip_special_tokens=True))
validate_specs(specs)
audio = render_from_specs(specs)
ipd.Audio(audio, rate=44100)

In [15]:
type(specs)

dict